In [7]:
# Download the required jar library for elastic


In [8]:
! sudo curl https://repo1.maven.org/maven2/org/elasticsearch/elasticsearch-spark-20_2.12/7.15.0/elasticsearch-spark-20_2.12-7.15.0.jar --output /usr/local/spark/jars/elasticsearch-spark-20_2.12-7.15.0.jar

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 2052k  100 2052k    0     0  9121k      0 --:--:-- --:--:-- --:--:-- 9121k


In [1]:
import pyspark
from pyspark.sql import SparkSession

In [2]:
#   .config("spark.es.nodes.wan.only","true") \
spark = SparkSession.builder \
    .master("local") \
    .appName('jupyter-pyspark') \
    .config("spark.es.nodes","es01") \
    .config("spark.es.port","9200") \
    .getOrCreate()
sc = spark.sparkContext
sc.setLogLevel("ERROR")

21/09/24 15:11:43 WARN NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Using Spark's default log4j profile: org/apache/spark/log4j-defaults.properties
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).


In [3]:
# let's get some file to shove into elastic
df = spark.read.csv("/home/jovyan/datasets/customers/customers.csv", header=True, inferSchema=True)
df.show()

+------+----------+--------------------+------+---------------+-----------+-----+------------+---------------+---------------+
| First|      Last|               Email|Gender|Last IP Address|       City|State|Total Orders|Total Purchased|Months Customer|
+------+----------+--------------------+------+---------------+-----------+-----+------------+---------------+---------------+
|    Al|    Fresco|  afresco@dayrep.com|     M|  74.111.18.161|   Syracuse|   NY|           1|             45|              1|
|  Abby|      Kuss|     akuss@rhyta.com|     F|  23.80.125.101|    Phoenix|   AZ|           1|             25|              2|
| Arial|     Photo|   aphoto@dayrep.com|     F|     24.0.14.56|     Newark|   NJ|           1|            680|              1|
| Bette|     Alott|    balott@rhyta.com|     F| 56.216.127.219|    Raleigh|   NC|           6|            560|             18|
| Barb |    Barion|bbarion@superrito...|     F|   38.68.15.223|     Dallas|   TX|           4|           1590| 

In [5]:
# write the file to the "customers" index with type "_doc"
df.write.format("es").save("customers/_doc")

In [9]:
# read the exact same data back out again
df2 = spark.read.format("es").load("customers/_doc")
df2.show()

+-----------+--------------------+------+------+----------+---------------+---------------+-----+------------+---------------+
|       City|               Email| First|Gender|      Last|Last IP Address|Months Customer|State|Total Orders|Total Purchased|
+-----------+--------------------+------+------+----------+---------------+---------------+-----+------------+---------------+
|   Syracuse|  afresco@dayrep.com|    Al|     M|    Fresco|  74.111.18.161|              1|   NY|           1|             45|
|    Phoenix|     akuss@rhyta.com|  Abby|     F|      Kuss|  23.80.125.101|              2|   AZ|           1|             25|
|     Newark|   aphoto@dayrep.com| Arial|     F|     Photo|     24.0.14.56|              1|   NJ|           1|            680|
|    Raleigh|    balott@rhyta.com| Bette|     F|     Alott| 56.216.127.219|             18|   NC|           6|            560|
|     Dallas|bbarion@superrito...| Barb |     F|    Barion|   38.68.15.223|              1|   TX|           4| 